<h1 style='font-size:40px'> HTTPS Setup with Kubernetes</h1>

<h2 style='font-size:30px'> 
    HTTPS Setup Overview
</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Para habilitarmos conexões HTTPS em nossa aplicação, nosso site deverá ter um certificado que habilite o uso desse protocolo. Usaremos o serviço LetsEncrypt para obtê-lo.
        </li>
        <li>
            De maneira geral, quando invocarmos o LetsEncrypt, este fará uma requisição a um endereço específico do nosso site. Caso esta seja bem sucedida, receberemos o certificado.
        </li>
    </ul>
 </div>

<figure>
        <center style='margin-top:20px'>
            <img src='img/17_https_flow.png'>
                <figcaption> Ilustração do ciclo de obtenção do certificado.</figcaption>
        </center>
    </figure>

<h2 style='font-size:30px'> 
    Domain Purchase
</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Aula em que compramos um nome de domínio para o nosso site. Em seguida, configuramos o site para apontar para o IP de nosso cluster.
        </li>
    </ul>
 </div>

In [2]:
! git add .
! git commit -m ' Aula 294'
! git push

[master 1423158]  Aula 292
 3 files changed, 64 insertions(+), 24 deletions(-)
 create mode 100644 img/17_https_flow.png
Enumerating objects: 10, done.
Counting objects: 100% (10/10), done.
Delta compression using up to 24 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 86.53 KiB | 43.26 MiB/s, done.
Total 6 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/felipesveiga/docker.git
   1e19727..1423158  master -> master


<p style='color:red'> Vi Aula 292 e 293; Aula 294</p>

<p style='color:red'> Visite a Aula 307, caso deseje deletar o seu cluster!</p>